In [1]:
from PIL import Image
import os
import re

import matplotlib.pyplot as plt
import numpy as np

import pylab as pl
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
from sklearn.svm import SVC

from skimage.feature import hog

In [2]:
folder='DataSet/' # location of training images
images = []
i=0

X=[]
Y=[]

for filename in os.listdir(folder):   # run the loop for all files in training folder
    img = Image.open(os.path.join(folder,filename)) # open image 

    img=img.resize([32,64])  # resize image  to a fixed size
    
    fd,hog_image = hog(img, orientations=9, pixels_per_cell=(4,4), cells_per_block=(2, 2), visualize=True, multichannel=True) # obtain HOG features
    
    X.append(fd)

    # obtain training labels from file name
    
    if re.match("Ross*", filename):
      Yt=0
    elif re.match("Gunther*", filename):
      Yt=1
    elif re.match("Joey*", filename):
      Yt=2
    elif re.match("Rachel*", filename):
      Yt=3
    else:
      Yt=4

    Y.append(Yt)
    
    i=i+1


In [3]:
fd.shape

(3780,)

In [4]:
target_names=['RossGeller', 'Gunther', 'JoeyTribbiani', 'PhoebeBuffay']

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [6]:
n_components = 16
pca = PCA(n_components=n_components, whiten=True).fit(X_train)

In [7]:
# project training and test data to lower dimension 
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

In [8]:
# Save PCA coefficients for use at prediction

import pickle as pk
pk.dump(pca, open("PCA_Coefficient.pkl","wb"))

In [9]:
print("Fitting the classifier to the training set")
param_grid = {
         'C': [1e3, 5e3, 1e4, 5e4, 1e5],
     
          
    'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1],
          }
clf = GridSearchCV(SVC(kernel='rbf', class_weight='balanced',probability=True), param_grid)
clf = clf.fit(X_train_pca, y_train)
print("Best estimator found by grid search:")
print(clf.best_estimator_)

Fitting the classifier to the training set
Best estimator found by grid search:
SVC(C=1000.0, class_weight='balanced', gamma=0.1, probability=True)


In [10]:
print(clf)

GridSearchCV(estimator=SVC(class_weight='balanced', probability=True),
             param_grid={'C': [1000.0, 5000.0, 10000.0, 50000.0, 100000.0],
                         'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1]})


In [11]:
# Save best SVM classifier weights for use at prediction

import joblib
joblib.dump(clf.best_estimator_, 'SVM_Classifier.pkl', compress = 1) # Only best parameters

['SVM_Classifier.pkl']

In [12]:
y_pred = clf.predict(X_test_pca)
y_pred

array([1, 0, 0, 4, 0, 1, 1, 0, 4, 4, 4, 0, 2, 0, 2, 1, 2, 1, 2, 2, 0, 4,
       0, 1, 1, 4, 2, 1, 0, 0, 2, 2, 4, 0, 2, 1, 0])

In [13]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.67      1.00      0.80         8
           1       1.00      1.00      1.00         9
           2       1.00      0.82      0.90        11
           4       1.00      0.78      0.88         9

    accuracy                           0.89        37
   macro avg       0.92      0.90      0.89        37
weighted avg       0.93      0.89      0.90        37

